In [ ]:
!pip install astroquery


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive


In [ ]:
print(NasaExoplanetArchive)


In [ ]:
exoplanets = NasaExoplanetArchive.query_criteria(
    table="pscomppars",
    select="pl_name, pl_orbper, pl_rade, st_teff, st_rad, sy_vmag",
    where="pl_orbper IS NOT NULL AND pl_rade IS NOT NULL"
)

df = exoplanets.to_pandas()
df.head()


In [ ]:
df.shape


In [ ]:
df.columns


In [ ]:
df.isnull().sum()


In [ ]:
plt.figure(figsize=(6,4))
plt.hist(df["pl_orbper"], bins=50)
plt.xscale("log")
plt.xlabel("Orbital Period (days)")
plt.ylabel("Number of planets")
plt.title("Distribution of Orbital Periods")
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
plt.hist(df["pl_rade"], bins=50)
plt.xscale("log")
plt.xlabel("Planet Radius (Earth radii)")
plt.ylabel("Number of planets")
plt.title("Distribution of Planet Radii")
plt.tight_layout()
plt.show()


In [ ]:
import os

os.makedirs("../data/raw", exist_ok=True)


In [ ]:
df.to_csv("../data/raw/confirmed_exoplanets.csv", index=False)


In [ ]:
import os
os.listdir("../data/raw")


## Adding TESS Objects of Interest (TOIs)

Confirmed planets represent successful detections.
To understand selection effects, we now include TESS Objects of Interest (TOIs),
which include both planet candidates and false positives.


In [ ]:
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive

toi = NasaExoplanetArchive.query_criteria(
    table="toi",
    select="toi, pl_orbper, pl_rade, tfopwg_disp",
    where="pl_orbper IS NOT NULL"
)

toi_df = toi.to_pandas()
toi_df.head()


In [ ]:
toi_df.shape


In [ ]:
toi_df["tfopwg_disp"].value_counts()


In [ ]:
confirmed = toi_df[toi_df["tfopwg_disp"] == "CP"]
candidates = toi_df[toi_df["tfopwg_disp"] == "PC"]

plt.figure(figsize=(6,4))
plt.scatter(candidates["pl_orbper"], candidates["pl_rade"],
            s=5, alpha=0.4, label="Candidates")
plt.scatter(confirmed["pl_orbper"], confirmed["pl_rade"],
            s=10, alpha=0.7, label="Confirmed")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Orbital Period (days)")
plt.ylabel("Planet Radius (Earth radii)")
plt.legend()
plt.title("TESS Candidates vs Confirmed Planets")
plt.tight_layout()
plt.show()


In [ ]:
plt.savefig("filename.png", dpi=200, bbox_inches="tight")


In [ ]:
import os
os.makedirs("../data/raw", exist_ok=True)
toi_df.to_csv("../data/raw/toi_catalog.csv", index=False)


In [ ]:
import numpy as np

# define radius bins (Earth radii)
radius_bins = np.logspace(np.log10(0.5), np.log10(20), 8)
radius_bins


In [ ]:
# prepare arrays
bin_centers = []
confirmation_fraction = []

for i in range(len(radius_bins) - 1):
    r_min = radius_bins[i]
    r_max = radius_bins[i+1]

    # candidates in bin
    cand_bin = toi_df[
        (toi_df["pl_rade"] >= r_min) &
        (toi_df["pl_rade"] < r_max)
    ]

    # confirmed in bin
    conf_bin = cand_bin[cand_bin["tfopwg_disp"] == "CP"]

    if len(cand_bin) > 0:
        frac = len(conf_bin) / len(cand_bin)
    else:
        frac = np.nan

    bin_centers.append(np.sqrt(r_min * r_max))
    confirmation_fraction.append(frac)


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(bin_centers, confirmation_fraction, marker="o")
plt.xscale("log")
plt.xlabel("Planet Radius (Earth radii)")
plt.ylabel("Confirmation Fraction")
plt.title("TESS Confirmation Efficiency vs Planet Radius")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("filename.png", dpi=200, bbox_inches="tight")
plt.show()


In [ ]:
confirmed_planets = df.copy()

obs_counts, _ = np.histogram(
    confirmed_planets["pl_rade"],
    bins=radius_bins
)


In [ ]:
eff = np.array(confirmation_fraction)

# avoid division by zero
corrected_counts = obs_counts / eff


In [ ]:
plt.figure(figsize=(6,4))
plt.step(radius_bins[:-1], obs_counts, where="post", label="Observed")
plt.step(radius_bins[:-1], corrected_counts, where="post", label="Bias-corrected")

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Planet Radius (Earth radii)")
plt.ylabel("Number of Planets")
plt.legend()
plt.title("Observed vs Bias-Corrected Planet Radius Distribution")
plt.tight_layout()
plt.savefig("filename.png", dpi=200, bbox_inches="tight")
plt.show()


### Interpretation and Caveats

The bias-corrected distribution shows a strong increase in the inferred
number of small planets, consistent with known detection and confirmation
biases in transit surveys. At large radii, the correction amplifies noise
due to low confirmation efficiency, indicating the need for regularization
and uncertainty modeling in future work.


In [ ]:
import numpy as np

eff = np.array(confirmation_fraction)

# number of candidates and confirmed per bin
n_cand = []
n_conf = []

for i in range(len(radius_bins) - 1):
    r_min = radius_bins[i]
    r_max = radius_bins[i+1]

    cand_bin = toi_df[
        (toi_df["pl_rade"] >= r_min) &
        (toi_df["pl_rade"] < r_max)
    ]
    conf_bin = cand_bin[cand_bin["tfopwg_disp"] == "CP"]

    n_cand.append(len(cand_bin))
    n_conf.append(len(conf_bin))

n_cand = np.array(n_cand)
n_conf = np.array(n_conf)


In [ ]:
eff_err = np.sqrt(eff * (1 - eff) / n_cand)


In [ ]:
corrected_err = corrected_counts * (eff_err / eff)


In [ ]:
plt.figure(figsize=(6,4))

plt.errorbar(
    bin_centers,
    corrected_counts,
    yerr=corrected_err,
    fmt="o",
    label="Bias-corrected (with uncertainty)"
)

plt.xscale("log")
plt.yscale("log")
plt.xlabel("Planet Radius (Earth radii)")
plt.ylabel("Corrected Planet Count")
plt.title("Bias-Corrected Radius Distribution with Uncertainty")
plt.legend()
plt.tight_layout()
plt.savefig("filename.png", dpi=200, bbox_inches="tight")
plt.show()


### Uncertainty Treatment

Confirmation efficiency uncertainties were estimated assuming binomial
statistics. These uncertainties were propagated into the bias-corrected
planet counts, revealing increased variance in regions of low detection
efficiency.


In [ ]:
# period bins (days)
period_bins = np.logspace(np.log10(0.5), np.log10(500), 8)
period_centers = np.sqrt(period_bins[:-1] * period_bins[1:])


In [ ]:
period_eff = []
period_eff_err = []

for i in range(len(period_bins) - 1):
    p_min = period_bins[i]
    p_max = period_bins[i+1]

    cand_bin = toi_df[
        (toi_df["pl_orbper"] >= p_min) &
        (toi_df["pl_orbper"] < p_max)
    ]

    conf_bin = cand_bin[cand_bin["tfopwg_disp"] == "CP"]

    if len(cand_bin) > 0:
        eff = len(conf_bin) / len(cand_bin)
        err = np.sqrt(eff * (1 - eff) / len(cand_bin))
    else:
        eff = np.nan
        err = np.nan

    period_eff.append(eff)
    period_eff_err.append(err)

period_eff = np.array(period_eff)
period_eff_err = np.array(period_eff_err)


In [ ]:
plt.figure(figsize=(6,4))

plt.errorbar(
    period_centers,
    period_eff,
    yerr=period_eff_err,
    fmt="o"
)

plt.xscale("log")
plt.xlabel("Orbital Period (days)")
plt.ylabel("Confirmation Efficiency")
plt.title("TESS Confirmation Efficiency vs Orbital Period")
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig("filename.png", dpi=200, bbox_inches="tight")
plt.show()


In [ ]:
# 2D bins
radius_bins_2d = np.logspace(np.log10(0.5), np.log10(20), 7)
period_bins_2d = np.logspace(np.log10(0.5), np.log10(500), 7)


In [ ]:
eff_2d = np.zeros((len(radius_bins_2d)-1, len(period_bins_2d)-1))
eff_2d[:] = np.nan

for i in range(len(radius_bins_2d)-1):
    for j in range(len(period_bins_2d)-1):
        r_min, r_max = radius_bins_2d[i], radius_bins_2d[i+1]
        p_min, p_max = period_bins_2d[j], period_bins_2d[j+1]

        cand_bin = toi_df[
            (toi_df["pl_rade"] >= r_min) &
            (toi_df["pl_rade"] < r_max) &
            (toi_df["pl_orbper"] >= p_min) &
            (toi_df["pl_orbper"] < p_max)
        ]

        conf_bin = cand_bin[cand_bin["tfopwg_disp"] == "CP"]

        if len(cand_bin) > 5:  # avoid noisy bins
            eff_2d[i, j] = len(conf_bin) / len(cand_bin)


In [ ]:
plt.figure(figsize=(7,5))
plt.imshow(
    eff_2d,
    origin="lower",
    aspect="auto",
    extent=[
        np.log10(period_bins_2d[0]),
        np.log10(period_bins_2d[-1]),
        np.log10(radius_bins_2d[0]),
        np.log10(radius_bins_2d[-1]),
    ],
    cmap="viridis"
)

plt.colorbar(label="Confirmation Efficiency")
plt.xlabel("log Orbital Period (days)")
plt.ylabel("log Planet Radius (Earth radii)")
plt.title("TESS Confirmation Efficiency Map")
plt.tight_layout()
plt.savefig("filename.png", dpi=200, bbox_inches="tight")
plt.show()
